In [226]:
#Importing packages
from selenium import webdriver
import pandas as pd

In [147]:
#change directory where you put your chrome driver
driver = webdriver.Chrome('/Users/tsblo/MSBA Fall/Text Analytics/selenium/chromedriver_win32/chromedriver.exe')

In [222]:
driver.get('https://www.beeradvocate.com/beer/top-rated/')

In [228]:
%%time

# create beer list for web scraper
beer_list = []

x = 2
while x < 252:
    beer = driver.find_elements_by_xpath("//*[@id='ba-content']/table/tbody/tr[{}]/td[2]/a/b".format(x))[0].text
    beer_list.append(beer)
    x += 1

Wall time: 7.46 s


In [229]:
len(beer_list)

250

In [230]:
%%time
all_comments = pd.DataFrame(columns = ['Product_name','product_review', 'user_ratings', "overall_rating"]) 

page = 0

for beer in beer_list:
    print(page)
    
    driver.find_element_by_link_text("{}".format(beer)).click()
    driver.implicitly_wait(1)
     
    ids = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/div/span/a[1]")
    
    rating = 0
    
    # not all beers have 25 reviews
    while rating <= (len(ids) - 1):

        product = beer
        
        # this gets the whole review with ratings and date, cant seem to figure out how to just get review
        review = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']")[rating].text
        
        # this gets all of the ratings, dont exactly know which rating he wants 
        user_ratings = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/span[4]")[rating].text
        
        # this gets the big overall rating, im guessing this is what rating he wants us to use
        overall_rating = driver.find_elements_by_xpath("//*[@id='rating_fullview_content_2']/span[1]")[rating].text

        all_comments.loc[len(all_comments)] = [product,review,user_ratings,overall_rating]

        rating += 1
    
    page += 1
    driver.execute_script("window.history.go(-1)")
    driver.implicitly_wait(3)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
Wall time: 35min 5s


In [231]:
def clean_review(s):
    string = s.splitlines()
    string = list(filter(None, string))
    del string[0:2]
    del string[-2:]
    return string

In [232]:
all_comments["clean_review"] = all_comments["product_review"].map(clean_review)

In [233]:
all_comments.shape

(6039, 5)

In [234]:
all_comments.tail()

,Product_name,product_review,user_ratings,overall_rating,clean_review
6034,Sure Bet,4.34/5 rDev -2.9%\nlook: 4.5 | smell: 4.25 | ...,look: 4.5 | smell: 4.25 | taste: 4.25 | feel: ...,4.34,"[Love their beers, they play more on rhe malt ..."
6035,Sure Bet,4.46/5 rDev -0.2%\nlook: 4.25 | smell: 4.5 | ...,look: 4.25 | smell: 4.5 | taste: 4.5 | feel: 4...,4.46,"[A - Pours a bright opaque orange with a 1/4"" ..."
6036,Sure Bet,4.54/5 rDev +1.6%\nlook: 4.75 | smell: 4.5 | ...,look: 4.75 | smell: 4.5 | taste: 4.5 | feel: 4...,4.54,[Appearance - The beer poured a murky orange-y...
6037,Sure Bet,4.38/5 rDev -2%\nlook: 4.5 | smell: 4.5 | tas...,look: 4.5 | smell: 4.5 | taste: 4.25 | feel: 4...,4.38,[Hazy faded gold with a fluffy white head. Sme...
6038,Sure Bet,4.31/5 rDev -3.6%\nlook: 4.25 | smell: 4.5 | ...,look: 4.25 | smell: 4.5 | taste: 4.25 | feel: ...,4.31,[Straw color with white head that dissipates q...


In [236]:
beer_list[249]

'Sure Bet'

In [237]:
all_comments.to_csv("6039 beer reviews of 250 beers.csv", sep="\t")